In [ ]:
# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 26/02/2024

In [ ]:
# There is a bug with the current version of ImageD11 in the site-wide Jupyter env.
# This has been fixed here: https://github.com/FABLE-3DXRD/ImageD11/commit/4af88b886b1775585e868f2339a0eb975401468f
# Until a new release has been made and added to the env, we need to get the latest version of ImageD11 from GitHub
# Put it in your home directory
# USER: Change the path below to point to your local copy of ImageD11:

import os

username = os.environ.get("USER")

id11_code_path = f"/home/esrf/{username}/Code/ImageD11"

import sys

sys.path.insert(0, id11_code_path)

In [ ]:
# import functions we need

import concurrent.futures
import timeit
import glob
import pprint
from shutil import rmtree
import time
from functools import partial

import matplotlib
%matplotlib ipympl

import h5py
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

import ImageD11.columnfile
# from ImageD11.sinograms.dataset import DataSet
from ImageD11.sinograms import properties, roi_iradon, dataset
# from ImageD11.blobcorrector import eiger_spatial
from ImageD11.grain import grain

import ImageD11.nbGui.nb_utils as utils


In [ ]:
# NOTE: For old datasets before the new directory layout structure, we don't distinguish between RAW_DATA and PROCESSED_DATA

### USER: specify your experimental directory

rawdata_path = "/home/esrf/james1997a/Data/ihma439/id11/20231211/RAW_DATA"

!ls -lrt {rawdata_path}

### USER: specify where you want your processed data to go

processed_data_root_dir = "/home/esrf/james1997a/Data/ihma439/id11/20231211/PROCESSED_DATA/James/20240226"

In [ ]:
samples_dict = {}

skips_dict = {
    "FeAu_0p5_tR_nscope": ["top_100um", "top_-100um"]
}

dset_prefix = "top"

sample_list = ["FeAu_0p5_tR_nscope"]

for sample in sample_list:
    all_dset_folders_for_sample = os.listdir(os.path.join(rawdata_path, sample))
    dsets_list = []
    for folder in all_dset_folders_for_sample:
        if dset_prefix in folder:
            dset_name = folder.split(f"{sample}_")[1]
            if dset_name not in skips_dict[sample]:
                dsets_list.append(dset_name)

    samples_dict[sample] = dsets_list
    
# manually override:

samples_dict = {"FeAu_0p5_tR_nscope": ["top_150um", "top_200um", "top_250um"]}

In [ ]:
def read_grains(ds):
    with h5py.File(ds.grainsfile, 'r') as hin:
        grp = hin['slice_recon']
        
        raw_intensity_array = grp['intensity'][:]
        grain_labels_array = grp['labels'][:]
       
        grains_group = 'grains'
        
        grains = []
        for gid_string in tqdm(sorted(hin[grains_group].keys(), key=lambda x: int(x))):
            gg = hin[grains_group][gid_string]
            ubi = gg.attrs['ubi'][:]
            g = ImageD11.grain.grain(ubi)
            # general grain properties
            g.gid = int(gid_string)
            g.translation = gg['translation'][:]
            g.cen = gg.attrs['cen']
            g.y0 = gg.attrs['y0']
            # sinogram stuff
            g.ssino = gg['ssino'][:]
            g.sinoangles = gg['sinoangles'][:]
            # reconstructions
            g.og_recon = gg['og_recon'][:]
            g.recon = gg['recon'][:]
            grains.append(g)
    
    return grains, raw_intensity_array, grain_labels_array

In [ ]:
z_translation_motor = "pz"

ds_dict = []

for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
        if not os.path.exists(dset_path):
            print(f"Missing DataSet file for {dataset} in sample {sample}, skipping")
            continue
        
        print("Importing DataSet object")
        
        ds = ImageD11.sinograms.dataset.load(dset_path)
        
        print("Importing grains")
        
        ds.grains, ds.raw_intensity_array, ds.grain_labels_array = read_grains(ds)
        
        print("Importing z positions")
        
        with h5py.File(ds.masterfile, 'r' ) as hin:
            this_z_trans_value = hin["1.1/instrument/positioners"][z_translation_motor][()]
            ds.zpos = this_z_trans_value  # this is in microns for samtz